In [4]:
import numba
import numpy as np
import math

In [5]:
kT = 2 / math.log(1 + math.sqrt(2), math.e)

@numba.jit(nopython=True)
def update_one_element(x, i, j):
    n, m = x.shape
    assert n > 0
    assert m > 0
    dE = 2 * x[i, j] * (
                     x[(i-1)%n, (j-1)%m]
                   + x[(i-1)%n,  j     ]
                   + x[(i-1)%n, (j+1)%m]

                   + x[ i     , (j-1)%m]
                   + x[ i     , (j+1)%m]

                   + x[(i+1)%n, (j-1)%m]
                   + x[(i+1)%n,  j     ]
                   + x[(i+1)%n, (j+1)%m]
                   )
    if dE <= 0 or exp(-dE / kT) > np.random.random():
        x[i, j] = -x[i, j]

@numba.jit(nopython=True)
def update_one_frame(x):
    n, m = x.shape
    for i in range(n):
        for j in range(0, m, 2):  # Even columns first to avoid overlap
            update_one_element(x, j, i)
    for i in range(n):
        for j in range(1, m, 2):  # Odd columns second to avoid overlap
            update_one_element(x, j, i)